### Initialization

In [1]:
import pandas
import time
import fasttext

### Contracts

In [54]:
import psycopg2
psycopg2_conn = psycopg2.connect(dbname='public_contracts', user='postgres', password='admin', host='localhost', port='5432')

In [3]:
from utils.document_processing import *

In [4]:
loader = DatabaseContractsLoader(psycopg2_conn)
loader.load_documents()
loader.prepare_documents()
loader.prepare_contracts()
df_contracts = pandas.DataFrame.from_dict(loader._contracts, orient='index')
df_contracts

Running query: select * from document where processed=True
Preparing total 1008 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 90.0%
Progress: 100.0%
Preparing total 159 contracts
Progress: 0.0%
Progress: 10.0%
Progress: 19.0%
Progress: 29.0%
Progress: 38.0%
Progress: 48.0%
Progress: 57.0%
Progress: 67.0%
Progress: 76.0%
Progress: 85.0%
Progress: 95.0%


,docs,text
2,"[{'id': 1, 'text': ' SMLOUVA O PROVÁDĚNÍ...",\n<FILE id=1>\n\n \n\n \n\nSMLOUVA O PROVÁDĚNÍ...
3,"[{'id': 3, 'text': ' SMLOUVA © DÍLO č.j....",\n<FILE id=3>\n\n \n\n \n\nSMLOUVA © DÍLO\nč.j...
6,"[{'id': 26, 'text': ' SMLOUVA O PROVÁDĚN...",\n<FILE id=26>\n\n \n\n \n\nSMLOUVA O PROVÁDĚN...
10,"[{'id': 35, 'text': 'N Á V R H SMLOUVA O DÍ...",\n<FILE id=35>\nN Á V R H\n\nSMLOUVA O DÍLO\...
7,"[{'id': 27, 'text': ' SMLOUVA © DÍLO č.j...",\n<FILE id=27>\n\n \n\n \n\nSMLOUVA © DÍLO\nč....
...,...,...
839,"[{'id': 4117, 'text': 'EVROPSKÁ UNIE 6 9 EVROP...",\n<FILE id=4117>\nEVROPSKÁ UNIE 6 9\nEVROPSKÝ ...
840,"[{'id': 4120, 'text': 'c o X bi EVROPSKÁ UNIE ...",\n<FILE id=4120>\nc o X\nbi EVROPSKÁ UNIE . 20...
841,"[{'id': 4121, 'text': '. a EVROPSKÁ UNIE . G0...",\n<FILE id=4121>\n.\n\na EVROPSKÁ UNIE .\nG0. ...
842,"[{'id': 4124, 'text': 'EVROPSKÁ UNIE p »007-1...",\n<FILE id=4124>\nEVROPSKÁ UNIE p\n\n»007-12\n...


### Subject

In [4]:
from utils.subject_extraction import *
from utils.context_extraction import *
from utils.subject_context_preprocessing import *
from utils.conllu_preprocessing import *

In [6]:
extractor = SubjectExtractor()

In [7]:
from udapi.core.document import Document
from ufal.udpipe import *
udpipe_model = Model.load("../model/udpipe/udpipe-ud-2.5-191206/czech-pdt-ud-2.5-191206.udpipe")

In [8]:
udp_pipeline = Pipeline(udpipe_model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")

In [9]:
import ufal.udpipe

class ComplexSubjectExtractor(SubjectExtractor):
    
    def __init__(self,
                 subj_context_extractor=None,
                 subj_context_preprocessor=None,
                 attributes_extractor=None,
                 text_annotator=None,
                 udapi_doc_creator=None,
                 conllu_attributes_preprocessor=None,
                 udapi_to_str_transformer=None,
                 items_tagger=None,
                 attribute_merger=None,
                 tag_cleaner=None,
                 concatenator=None):
        self._subj_context_extractor = subj_context_extractor if subj_context_extractor is not None else \
            AdvancedSubjectContextExtractor()
        self._subj_context_preprocessor = subj_context_preprocessor if subj_context_preprocessor is not None else \
            SubjectContextPreprocessor()
        self._attribures_extractor = attributes_extractor if attributes_extractor is not None else \
            SubjectContextPreprocessor(transformers=[AttributeExtractor(keep_text=False, keep_attributes=True)])
        self._text_annotator = text_annotator if text_annotator is not None else \
            TextAnnotator()
        self._udapi_doc_creator = udapi_doc_creator if udapi_doc_creator is not None else \
            UdapiFromConlluTransformer()
        self._conllu_attributes_preprocessor = conllu_attributes_preprocessor if conllu_attributes_preprocessor is not None else \
            ConlluSubjectContextPreprocessor()
        self._udapi_to_str_transformer = udapi_to_str_transformer if udapi_to_str_transformer is not None else \
            UdapiToStrTransformer()
        self._items_tagger = items_tagger if items_tagger is not None else \
            SubjectContextPreprocessor(transformers=[AttributeTagger(attr_tag='<ITEM>;<ITEM/>', keep_text=False)])
        self._attribute_merger = attribute_merger if attribute_merger is not None else \
            AttributeMerger()
        self._tag_cleaner = tag_cleaner if tag_cleaner is not None else \
            SubjectContextPreprocessor(transformers=[AttributeTagCleaner(attr_pattern=r'<[A-Z_]+>(.*)<[A-Z_]+/>')])
        self._concatenator = concatenator if concatenator is not None else \
            AttributeConcatenator()

    def extract(self, text):
        subj_context = self._subj_context_extractor.process(text, True)
        filtered_context = self._subj_context_preprocessor.process(subj_context, True)
        attributes = self._attribures_extractor.process(filtered_context, True)
        decomposition = self._text_annotator.process(filtered_context, True)
        udapi_doc = self._udapi_doc_creator.process(decomposition, True)
        filtered_doc = self._conllu_attributes_preprocessor.process(udapi_doc, True)
        filtered_doc_text = self._udapi_to_str_transformer.process(filtered_doc, True)
        attributes2 = self._items_tagger.process(filtered_doc_text, True)
        merged_attributes = self._attribute_merger.process(list(zip(attributes, attributes2)), True)
        subject_items = self._tag_cleaner.process(merged_attributes, True)
        subject = self._concatenator.process(subject_items)
        return subject

In [10]:
subj_context_transformers = [
    NumeralLinesFilter(too_many_numerals_ratio_threshold=0.5),
    TooShortLinesFilter(too_short_line_threshold=5),
    IrrelevantLinesFilter(keywords=['strana', 'stránka', 'e-mail'], max_line_length=75, lower=True),
    IrrelevantLinesFilter(keywords=['Tel:', 'Fax:', 'IČ:', 'IČO:', 'DIČ:'], max_line_length=75,
                          lower=False),
    IrrelevantLinesRegexFilter(patterns=[r'www', r'[\w\-\.]+\s*@\s*([\w\-]+\.)+[\w\-]{2,4}']),  # email
    IrrelevantLinesRegexFilter(patterns=[r'(\+\d{2,3}){0,1}(\s{0,1}\d{3}){3}']),  # phone
    RegexReplaceTransformer(pattern_to_transform=r',([\s]+[A-Z][a-z ])', result_pattern='.\g<1>'),  # . vs , correction
    RegexReplaceTransformer(pattern_to_transform=r'\n[ \t]*([^\d]{0,1}[\d]{1,2}[^\d])+[ \t]*',  # paragraph numbers
                            result_pattern='\n'),
    BlankLinesFilter(replacement='\n', top_n_frequency=200, top_n_var_threshold=5,
                     full_line_threshold=0.85, min_max_line_length=0),
    ReplaceMarksTransformer(marks_to_transform='„“', result_mark='"'),
    TooLongLinesTransformer(forbidden_delimiters='aábcčdďeéěfghiíjklmnňoópqrřsštťuúůvwxyýzž0123456789',
                            special_delimiters={'-': (r'[\s,\.](-)[\s]+[^(Kč)]', 1)},
                            too_long_line_treshold=200),
    RegexReplaceTransformer(pattern_to_transform=r'\([^\n()]*\)', result_pattern=''),  # bracket erasing
    RegexReplaceTransformer(pattern_to_transform=r'([^\n ])[ ]*\n', result_pattern='\g<1>.\n'),  # . filling
    RegexReplaceTransformer(pattern_to_transform=r'(([Nn]ázev|[Pp]opis)[^\n,.:"()]{5,})(\s[A-Z][^\n:]{10})',
                            result_pattern='\g<1>:\g<3>'),  # : filling
    ReplaceMarksTransformer(marks_to_transform=[':'], result_mark=':.'),
    ReplaceMarksTransformer(marks_to_transform=[';.', ',.'], result_mark='.'),
    ReplaceMarksTransformer(marks_to_transform=['..'], result_mark='.'),
    RegexReplaceTransformer(pattern_to_transform=r'[ ]*\.', result_pattern='.'),
]
subj_context_preprocessor = SubjectContextPreprocessor(transformers = subj_context_transformers)

attribute_transformers = [
    StructureItemEnumerationExtractor(),
    CharTupleItemEnumerationExtractor(),
    ItemColonExtractor(),
    HeaderItemEnumerationExtractor(),
    QuotedContractNameExtractor(),
    StructuredContractNameExtractor(),
    AttributeExtractor(keep_text=False, keep_attributes=True)
]
attributes_extractor = SubjectContextPreprocessor(transformers = attribute_transformers)

text_annotator = TextAnnotator(pipeline=udp_pipeline)

conllu_transformers = [
    UdapiWordOccurrencePartSentenceFilter(keywords=['cena', 'hodnota', 'DPH']),
    UdapiWordOccurrencePartSentenceFilter(keywords=['příloha', 'dále', 'jen']),
    NonSubjectPartSentenceFilter(),
    EmptyBundlesFilter(),
]
conllu_preprocessor = ConlluSubjectContextPreprocessor(transformers=conllu_transformers)

extractor = ComplexSubjectExtractor(subj_context_preprocessor=subj_context_preprocessor,
                                    attributes_extractor=attributes_extractor,
                                    text_annotator=text_annotator,
                                    conllu_attributes_preprocessor=conllu_preprocessor)

### Embedding 

In [5]:
import fasttext
import time

class FastTextEmbedder:
    
    def __init__(self, model='../model/fasttext/wiki.cs/wiki.cs.bin'):
        if isinstance(model, str):
            print('Loading FastText model from: ' + model)
            start = time.time()
            model = fasttext.load_model(model)
            end = time.time()
            print('Model loaded in: ' + str(end-start) + ' sec')
        if not isinstance(model, fasttext.FastText._FastText):
            raise ValueError('model must be ' + fasttext.FastText._FastText + ' or a path to fasttext binary model')
        self._model = model
    
    def embed(self, token):
        embedding = self._model.get_sentence_vector(token)
        return embedding

    def process(self, data):
        if isinstance(data, list):
            return [self.embed(text) for text in data]
        if isinstance(data, str):
            return self.embed(data)
        return None

In [6]:
%%time

model = fasttext.load_model('../model/fasttext/wiki.cs/wiki.cs.bin')

Wall time: 35.4 s


In [7]:
embedder = FastTextEmbedder(model)

### Playground 

In [14]:
%%time

df_contracts['subject'] = df_contracts['text'].apply(lambda text: extractor.extract(text))

AdvancedSubjectContextExtractor: Elapsed time: 0.7397 seconds
SubjectContextPreprocessor: Elapsed time: 0.5946 seconds
SubjectContextPreprocessor: Elapsed time: 0.0947 seconds
TextAnnotator: Elapsed time: 75.4505 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0523 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0748 seconds
UdapiToStrTransformer: Elapsed time: 0.0017 seconds
SubjectContextPreprocessor: Elapsed time: 0.0009 seconds
AttributeMerger: Elapsed time: 0.0002 seconds
SubjectContextPreprocessor: Elapsed time: 0.0002 seconds
AdvancedSubjectContextExtractor: Elapsed time: 0.2212 seconds
SubjectContextPreprocessor: Elapsed time: 0.0451 seconds
SubjectContextPreprocessor: Elapsed time: 0.0117 seconds
TextAnnotator: Elapsed time: 3.4994 seconds
UdapiFromConlluTransformer: Elapsed time: 0.1504 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.2253 seconds
UdapiToStrTransformer: Elapsed time: 0.0002 seconds
SubjectContextPreprocessor: Elapsed time: 0.0097 second

TextAnnotator: Elapsed time: 0.1732 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0039 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0188 seconds
UdapiToStrTransformer: Elapsed time: 0.0001 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
AttributeMerger: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
AdvancedSubjectContextExtractor: Elapsed time: 0.0194 seconds
SubjectContextPreprocessor: Elapsed time: 0.0035 seconds
SubjectContextPreprocessor: Elapsed time: 0.0025 seconds
SubjectContextPreprocessor: Elapsed time: 0.0008 seconds
SubjectContextPreprocessor: Elapsed time: 0.0024 seconds
TextAnnotator: Elapsed time: 0.1518 seconds
TextAnnotator: Elapsed time: 0.6975 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0025 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0121 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0200 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0712 seconds
UdapiT

ConlluSubjectContextPreprocessor: Elapsed time: 0.1195 seconds
UdapiToStrTransformer: Elapsed time: 0.0002 seconds
UdapiToStrTransformer: Elapsed time: 0.0001 seconds
SubjectContextPreprocessor: Elapsed time: 0.0001 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
AttributeMerger: Elapsed time: 0.0000 seconds
AttributeMerger: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0001 seconds
AdvancedSubjectContextExtractor: Elapsed time: 0.7172 seconds
SubjectContextPreprocessor: Elapsed time: 0.0093 seconds
SubjectContextPreprocessor: Elapsed time: 0.0014 seconds
SubjectContextPreprocessor: Elapsed time: 0.0012 seconds
SubjectContextPreprocessor: Elapsed time: 0.0104 seconds
SubjectContextPreprocessor: Elapsed time: 0.0119 seconds
SubjectContextPreprocessor: Elapsed time: 0.0029 seconds
SubjectContextPreprocessor: Elapsed time: 0.0009 seconds
SubjectContextPreprocessor: Elapsed time: 0.0005 sec

AdvancedSubjectContextExtractor: Elapsed time: 0.0357 seconds
SubjectContextPreprocessor: Elapsed time: 0.0040 seconds
SubjectContextPreprocessor: Elapsed time: 0.0081 seconds
SubjectContextPreprocessor: Elapsed time: 0.0025 seconds
SubjectContextPreprocessor: Elapsed time: 0.0102 seconds
TextAnnotator: Elapsed time: 0.3248 seconds
TextAnnotator: Elapsed time: 1.4688 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0066 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0396 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0277 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.1382 seconds
UdapiToStrTransformer: Elapsed time: 0.0000 seconds
UdapiToStrTransformer: Elapsed time: 0.0001 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
AttributeMerger: Elapsed time: 0.0000 seconds
AttributeMerger: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
SubjectContextP

AdvancedSubjectContextExtractor: Elapsed time: 18.6880 seconds
SubjectContextPreprocessor: Elapsed time: 0.0043 seconds
SubjectContextPreprocessor: Elapsed time: 0.0037 seconds
SubjectContextPreprocessor: Elapsed time: 0.0027 seconds
SubjectContextPreprocessor: Elapsed time: 0.0045 seconds
SubjectContextPreprocessor: Elapsed time: 0.0036 seconds
SubjectContextPreprocessor: Elapsed time: 0.0008 seconds
TextAnnotator: Elapsed time: 0.6897 seconds
TextAnnotator: Elapsed time: 0.6703 seconds
TextAnnotator: Elapsed time: 0.1762 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0175 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0161 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0029 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0537 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0562 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0179 seconds
UdapiToStrTransformer: Elapsed time: 0.0001 seconds
UdapiToStrTransformer: Elapsed time: 0.0001 seconds

...

TextAnnotator: Elapsed time: 0.2575 seconds
TextAnnotator: Elapsed time: 0.3688 seconds
TextAnnotator: Elapsed time: 0.1102 seconds
TextAnnotator: Elapsed time: 0.2456 seconds
TextAnnotator: Elapsed time: 0.3784 seconds
TextAnnotator: Elapsed time: 0.1188 seconds
TextAnnotator: Elapsed time: 0.3724 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0052 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0098 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0025 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0069 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0085 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0023 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0071 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0169 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0386 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0088 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0176 seconds
ConlluSubjectContextPreprocessor: Elapsed

TextAnnotator: Elapsed time: 1.7611 seconds
TextAnnotator: Elapsed time: 0.3666 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0395 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0068 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.1495 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0289 seconds
UdapiToStrTransformer: Elapsed time: 0.0000 seconds
UdapiToStrTransformer: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0001 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
AttributeMerger: Elapsed time: 0.0000 seconds
AttributeMerger: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
AdvancedSubjectContextExtractor: Elapsed time: 0.0401 seconds
SubjectContextPreprocessor: Elapsed time: 0.0087 seconds
SubjectContextPreprocessor: Elapsed time: 0.0012 seconds
TextAnnotator: Elapsed time: 0.3022 seconds
UdapiFromConlluTransformer: 

TextAnnotator: Elapsed time: 0.4815 seconds
TextAnnotator: Elapsed time: 0.3837 seconds
TextAnnotator: Elapsed time: 0.1399 seconds
TextAnnotator: Elapsed time: 0.0508 seconds
TextAnnotator: Elapsed time: 0.4018 seconds
TextAnnotator: Elapsed time: 0.1118 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0131 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0087 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0036 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0010 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0085 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0030 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0517 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0323 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0112 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0053 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0328 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0084 seconds
UdapiToStrTransf

TextAnnotator: Elapsed time: 0.2581 seconds
TextAnnotator: Elapsed time: 0.1343 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0077 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0029 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0257 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0137 seconds
UdapiToStrTransformer: Elapsed time: 0.0000 seconds
UdapiToStrTransformer: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
AttributeMerger: Elapsed time: 0.0000 seconds
AttributeMerger: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
AdvancedSubjectContextExtractor: Elapsed time: 0.0295 seconds
SubjectContextPreprocessor: Elapsed time: 0.0167 seconds
SubjectContextPreprocessor: Elapsed time: 0.0068 seconds
TextAnnotator: Elapsed time: 1.4916 seconds
UdapiFromConlluTransformer: 

TextAnnotator: Elapsed time: 0.2416 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0051 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0182 seconds
UdapiToStrTransformer: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
AttributeMerger: Elapsed time: 0.0000 seconds
SubjectContextPreprocessor: Elapsed time: 0.0000 seconds
AdvancedSubjectContextExtractor: Elapsed time: 0.0209 seconds
SubjectContextPreprocessor: Elapsed time: 0.0049 seconds
SubjectContextPreprocessor: Elapsed time: 0.0025 seconds
SubjectContextPreprocessor: Elapsed time: 0.0020 seconds
SubjectContextPreprocessor: Elapsed time: 0.0013 seconds
TextAnnotator: Elapsed time: 0.3233 seconds
TextAnnotator: Elapsed time: 0.2939 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0076 seconds
UdapiFromConlluTransformer: Elapsed time: 0.0058 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0370 seconds
ConlluSubjectContextPreprocessor: Elapsed time: 0.0250 seconds
UdapiT

In [15]:
%%time

df_contracts['subject_items'] = df_contracts['subject'].apply(lambda text: text.split('\n'))

Wall time: 3.01 ms


In [16]:
%%time

df_contracts['embedding'] = df_contracts['subject_items'].apply(lambda items: embedder.process(items))

Wall time: 45.9 s


In [122]:
class DBManager:
    
    def __init__(self, connection):
        self._connection = connection
        
    def runQuery(self, query):
        cursor = self._connection.cursor()
        cursor.execute(query)
        if 'select' in query.lower():
            data = cursor.fetchall()
        elif 'returning' in query.lower():
            data = cursor.fetchone()[0]
        else:
            data = True
        cursor.close()
        return data

class SubjectItemManager(DBManager):
    
    def __init__(self, connection):
        super().__init__(connection)
        self._load_query = 'select * from subject_item'
        
    def loadFromDB(self, parts=10):
        print("Running query: " + self._load_query)
        raw_data = self.runQuery(self._load_query)
        
        contract_items = {}
        total_items = len(raw_data)
        print("Loading total " + str(total_items) + " items")
        for i, item in enumerate(raw_data):
            if i % (int(total_items / parts)) == 0:
                print("Progress: {}%".format(numpy.ceil(i * 100 / total_items)))
            item_id = item[0]
            contract_id = item[1]
            item_desc = item[2]
            lembedding = item[3]
            embedding = numpy.array(lembedding)
            contract = contract_items.get(contract_id, {'contract_id':contract_id, 'subject_items':[],'embedding':[]})
            contract['subject_items'].append(item_desc)
            contract['embedding'].append(embedding)
            contract_items[contract_id] = contract
        return pandas.DataFrame(contract_items.values())
    
    def _truncateDB(self):
        self.runQuery('truncate table subject_item')

    def saveToDB(self, df_contract_items):
        self._truncateDB()
        for index, row in df_contract_items.iterrows():
            contract_id = index
            print(contract_id)
            subject_items = row['subject_items']
            embeddings = row['embedding']

            for i, item in enumerate(subject_items):
                print('    '+item)
                embedding = embeddings[i]
                lembedding = embedding.tolist()
                cursor = self._connection.cursor()

                postgres_insert_query = """INSERT INTO subject_item (contract_id, item_desc, embedding)
                                            VALUES (%s,%s,%s)"""
                record_to_insert = (contract_id, item, lembedding)
                cursor.execute(postgres_insert_query, record_to_insert)

                self._connection.commit()
                count = cursor.rowcount
                cursor.close()
                
simngr = SubjectItemManager(psycopg2_conn)

In [123]:
df_contract_items = simngr.loadFromDB()
df_contract_items

Running query: select * from subject_item
Loading total 2291 items
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 90.0%
Progress: 100.0%


,contract_id,subject_items,embedding
0,2,[krmení zvěře jádrovým a granulovým krmivem s ...,"[[0.00463546579703689, -0.079517237842083, 0.0..."
1,68,"[otovitel se zavazuje, že vyzve objednatele k ...","[[-0.00496369041502476, -0.0264993589371443, 0..."
2,23,[otovitel je povinen uchovávat veškerou dokume...,"[[-0.0100010493770242, -0.0357969366014004, 0...."
3,3,"[zhotovení díla, a to:. část č. 1. Oprava oplo...","[[-0.00123070902191103, -0.0510847456753254, 0..."
4,26,"[, VRSTVY PRO OBNOVU A OPRAVY Z ASF BETONU ACO...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...
154,839,"[Nové biotechnologické produkty ÚEB AVČR, reg....","[[-0.0160277131944895, -0.0321324989199638, 0...."
155,840,"[Nové biotechnologické produkty ÚEB AVČR, dodá...","[[-0.0254124347120523, -0.0627986714243889, 0...."
156,841,"[Nové biotechnologické produkty UEB AVČR, chem...","[[-0.0470611900091171, -0.0829258784651756, 0...."
157,842,"[Nové biotechnologické produkty ÚEB AVČR, chem...","[[-0.0254124347120523, -0.0627986714243889, 0...."


In [35]:
simngr.saveToDB(df_contract_items)

2
    krmení zvěře jádrovým a granulovým krmivem s využitím traktoru
    krmení černé zvěře
    rozvozu dikalcia a soli traktorem
    používání a využívání nakladače na seno
    úpravě cest
    vláčení
    sekaní
    zemědělských pracích
    provádění prací v Lánské oboře, zejména nakládání a zavážení krmiva a medikamentů pro lesní zvěř chovanou v Lánské oboře na jednotlivá krmná místa po stanovených krmných trasách dle určeného harmonogramu rozvozu krmení a medikamentů a také dalších prací
68
    otovitel se zavazuje, že vyzve objednatele k prověření prací , které v dalším postupu prací budou zakryty nebo se stanou nepřístupnými; výzva bude provedena zápisem s třídenním předstihem a musí být podepsána objednatelem, jestliže se objednatel k prověření prací  ve stanovené lhůtě nedostaví, ačkoliv byl k tomu řádně vyzván, je povinen hradit náklady dodatečného odkrytí, pokud takové odkrytí požaduje; zjistí-li se však při dodatečném odkrytí, že práce byly provedeny vadně, nese náklady dodat

    Rozsah prací je stanoven cenovou nabídkou v rozsahu položkového rozpočtu ze dne…………………. , který je přílohou č. 1 této smlouvy
    renovace toalety v salonu residence Maroka
7
    zhotovení díla, a to:. část č. 1. Oprava oplocení kolem hájovny Klíčava č
    prohlídku staveniště s jeho stavem plně seznámen dostatečná na kompletní dokončení díla tak, aby bylo plně způsobilé k plánovanému užívání, které je charakterizováno v zadávací dokumentaci a ve výkazu výměr
    k řádnému provedení díla
    zejména provedení těchto činností zhotovitelem:
    atestů, certifikátů, prohlášení o shodě, revizních zpráv a dalších dokladůkteré a užívání
    strojů a materiálů ponechaných na staveništi, provedení veškerých prostupů, otvorů, změn tvaru konstrukcí nutných pro ostatní dodávky a práce, a to vše v rámci koordinace s ostatními profesemi, náklady na media
    ji , pod vlastním jménem, na vlastní odpovědnost
    v takovém případě v tomto smyslu harmonogram postupu prací a takto upravený předat be

    b) Úklid společných prostor objektů spravovaných DS
    Václavské náměstí 1282/51. Praha 1
    Nad Výšinkou 125/7. Praha 5
    Na Baště sv. Jiří 258/7. Praha 6
    Na Zátorce 339/10. Praha 6
    Standardy a četnosti prací jsou uvedeny v příloze č. 4 této smlouvy - Standardy úklidových prací pro objekty DS
    Výměry a specifikace prací jsou uvedeny v příloze č. 3 této smlouvy – Cenová nabídka
    c)  Dodávka úklidových prostředků a hygienických potřeb pro objekt Václavské náměstí 816/49. Praha 1
    vztahy mezi smluvními stranami při zajišťování úklidových prací a zajištění dodávek čistících a hygienických prostředků v objektu Václavské náměstí 816
22
    Nátěry dveří, oken a okenic
    Rekonstrukce koupelen a WC
    Renovace parket
    Nátěr mříží a zábradlí
    Oprava dlažby na terase objektu nad garáží
    Oprava Soklu na objektu – vyspravení a nátěr
    Odstranění vlhkosti v objektu
    Dodávka a montáž kuchyní
    Dodávka a montáž klimatizace do podkroví objektu
    ·  Nátěry 

    obce s rozšířenou působností — řešení problémů, rady, datové služby
    každoroční zajištění aktuálního softwarového prostředku pro rezortní statistické zjišťování - realizací zakázky je zpřístupnění myslivecké statistiky pro odbornou veřejnost a zajištění konzultací na všech uživatelských úrovních při zachování návaznosti na stávající software
65
    — | dokumentace  v českém a anglickém jazyce
    — | dokumentace  v českém a anglickém jazyce
    v odvození parametrů rovnice mzdové dynamiky pro mikrosimulační model důchodového systému MPSV a závazek Objednatele
    provedení datové analýzy databáze ISPV spočívající v tvorbě datové základny , vyhodnocení možností využití zahraničních modelů, tak jak byly navrženy ve Studii proveditelnosti implementace rozhodovacích procesů do dynamického mikrosimulačního modelu důchodového systému
72
    NOVOSTAVBA DOMU PRO CHRÁNĚNÉ BYDLENÍ, HROTOVICE, P.Č. 3/1 A 9/11
    zajištění vytýčení veškerých inženýrských sítí, odpovědnost za jejich neporuš

    Magnetický kontakt pro hlídání dveří a oken DET MAG MAS303
    Propojovací krabice pro magnetické kontakty KRA RKZ111
    Čidlo detektoru kouřeDET POŽ SS2351NL OPTICKOKOUŘOVÝ
    Kabel KAB ZAB FI-H06
    Vícežilový kabel FTP Cat.5
    Lišta 20x40
    Demontážní a montážní práce, přeodporování detektorů
    Konfigurace systému PZTS, ASSET
    Podružný materiál
    jejich nevhodnost, která by překážela řádnému zahájení, provádění a dokončení
    v rozporu s platnými obecně závaznými právními předpisy, technickými předpisy, technickými pravidly nebo požadovanou úrovní výsledné kvality či parametry díla
    Oprava EZS centrálních pultů v objektech v péči PS 0705 Přáslavice
    Doba a místo plnění veřejné zakázky
    Oprava EZS centrálních pultů v objektech v péči PS 0705 Přáslavice
    neuvedeno
    
69
    Obnova chodníků — Nové Dvory
    a/ touto smlouvou, a
    b/ právními předpisy
    c/ podmínkami stanovenými ČSN
    zhotovení stavby dle projektové dokumentace pro provádění stavby

...

    kódu a názvu typové položky), je uvedena v příloze č. 2 této Smlouvy (Specifikace Zboží –
    kancelářské potřeby)
    Kontaktní osoby pro účely Smlouvy
    Kontaktní osobou Centrálního zadavatele pro účely této Smlouvy, neoznámí-li Centrální zadavatel
    odpovědné za převzetí Zboží na jednotlivých místech plnění jsou uvedeny v příloze č. 1 této
    Smlouvy
    Kontaktní osobou Prodávajícího pro účely této Smlouvy, neoznámí-li Prodávající Centrálnímu
    Práva a povinnosti Smluvních stran
    Smluvní strany se zavazují vzájemně spolupracovat a poskytovat si veškeré informace potřebné
    pro řádné plnění závazků této Smlouvy. Smluvní strany se dále zavazují vzájemně
    se informovat o veškerých skutečnostech, které jsou nebo mohou být důležité pro řádné plnění
    této Smlouvy
    Smluvní strany se zavazují plnit povinnosti vyplývající z této Smlouvy tak, aby nedocházelo
    k prodlení s plněním jednotlivých termínů a k prodlení s placením jednotlivých peněžních závazků
    Prodá

    Dodávání barev, laků a malířského materiálu
    neuvedeno  Doba a místo plnění veřejné zakázky
    Dodávání barev, laků a malířského materiálu
    neuvedeno  Doba a místo plnění veřejné zakázky
    
774
    EXTERNÍ POSKYTOVÁNÍ SLUŽEB KE ZPRACOVÁNÍ ŽÁDOSTÍ O KRÁTKODOBÁ VÍZA v turkmenistánu
    
775
    DKVM-222/2port USB KVM
    
717
    Implementace politiky stárnutí na krajskou úroveň
    na probíhající a očekávané demografické změny v souladu s výzvami
    provedení analýzy, ve které bude zmapována situace v oblasti stárnutí populace dle zadaných témat a okruhů
    především formulace tezí státní sociální politiky v oblasti přípravy na stárnutí a podpora její implementace na úroveň vyšších územně samosprávných celků a následně rovněž na úroveň samosprávy v ostatních zemích EU - charakterizována komplexním, nikoli striktně resortním přístupem
    na probíhající a očekávané demografické změny v souladu s výzvami
    provedení analýzy, ve které bude zmapována situace v oblasti stárn

    233223-8 - Obnova povrchu vozovky
    233290-8 - Instalace a montáž dopravního značení
    221119-9 - Stavební úpravy při rekonstrukci mostů
    Obecné vymezení předmětu veřejné zakázky
    Dle ustanovení § 26 a § 53 zákona se jedná o podlimitní veřejnou zakázku na stavební práce  s názvem "Silnice III/2923 Chuchelna - Palučiny"
    Klasifikace předmětu veřejné zakázky
    CPV veřejné zakázky
    45233141-9 - Práce na údržbě silnic
    45233223-8 - Obnova povrchu vozovky
    45233290-8 - Instalace a montáž dopravního značení
    45221119-9 - Stavební úpravy při rekonstrukci mostů
    50230000-6 - Opravy a údržba silnic a dalších zařízení a související služby
    Výzva k podání nabídky a zadávací dokumentace
    "Silnice III/2923 Chuchelna - Palučiny"
    rekonstrukce silnice III/2923 v obci Chuchelna a místní části města Semily
    Silnice III/2923 Chuchelna - Palučiny
    Silnice III/2923 Chuchelna - Palučiny
    Silnice III/2923 Chuchelna - Palučiny
    Rekonstrukce  silnice III/

    v součinnosti
    proti poškození
    stavbu v pracovní dny :.00 :.30
824
    
825
    zjištění skutečné hmotnostní vlhkosti zdiva v interiéru 1PP ÷ 1 NP a  exteriéru  v budově Rooseveltova a budově Husova
    stanovení obsahu výkvětotvorných solí  pro objekt  Rooseveltova a objekt Husova
    návrh sanace vlhkého zdiva
    zjištění způsobu provedení případných svislých hydroizolací
    zjištění kompletní skladby podlah a jejich kvalit z jádrových vývrtů vyjmutých  z konstrukce včetně případné hydroizolační vrstvy
    provedení celkové vizuální prohlídky objektů se zaměřením a zdokumentováním  viditelných vad a poruch stavebních konstrukcí způsobené vlhkostí
    provedení plošného průzkumu v objektech v dotčených podlažích pro komplexní  posouzení příčin vlhkosti a to i v místech, kde se vlhkost na první pohled vizuálně  neprojevuje
    schématické zakreslení postižených částí
    zpracování závěrečné zprávy stavebně technického průzkumu, sloužící jako podklad  pro budoucí projekční

843
    Nové biotechnologické produkty ÚEB AVČR
     ji EVROPSKÝ FOND PRO REGIONÁLNÍ ROZVOJ 
    i doprava do mista a dodání zákonných dokladů
    Nové biotechnologické produkty ÚEB AVČR
    dodávka chemikálií pro účely zajištění aktivit projektu "Nové biotechnologické produkty ÚEB AVČR
    na jednotlivé části, které mají stanoveny .
844
    Nové biotechnologické produkty ÚEB AVČR
    chemikálie pro účely zajištění aktivit projektu "Nové biotechnologické produkty ÚEB AVČR
    Nové biotechnologické produkty ÚEB AVČR
    dodávka chemikálií pro účely zajištění aktivit projektu "Nové biotechnologické produkty ÚEB AVČR
    na jednotlivé části, které mají stanoveny .
846
    Nové biotechnologické produkty ÚEB AVČR
    dodávka chemikálií pro účely zajištění aktivit projektu "Nové biotechnologické produkty ÚEB AVČR
    na jednotlivé části, které mají stanoveny .
847
    Nové biotechnologické produkty ÚEB AVČR
    chemikálie pro účely zajištění aktivit projektu "Nové biotechnologické produkty Ú

In [47]:
class UserProfile:
    
    def __init__(self, user_id=None, interest_items=[], embeddings=[]):
        self.user_id = user_id
        self.interest_items = interest_items
        self.embeddings = embeddings

    def __repr__(self):
        return 'UP'+str(self.user_id)+'('+str([(item, emb[:2]) for item, emb in zip(self.interest_items, self.embeddings)])+')'
        
class UserProfileFactory:
    
    def __init__(self, embedder):
        self._embedder=embedder
        
    def fromItems(self, items):
        print('Embedding total '+str(len(items))+' items')
        embeddings = self._embedder.process(items)
        profile = UserProfile(None, items, embeddings)
        return profile

In [49]:
upfac = UserProfileFactory(embedder)
interests = [['automobily'], ['silniční komunikace', 'stavby'], ['výpočetní technika'], ['politika']]
ups = [upfac.fromItems(intr) for intr in interests]
user_profile_items = [{'user_id':None, 'interest_items':up.interest_items, 'embeddings':up.embeddings} for up in ups]
df_user_profile = pandas.DataFrame(user_profile_items)
df_user_profile

Embedding total 1 items
Embedding total 2 items
Embedding total 1 items
Embedding total 1 items


,user_id,interest_items,embeddings
0,None,[automobily],"[[-0.022099614, -0.10972873, 0.13296181, 0.015..."
1,None,"[silniční komunikace, stavby]","[[-0.016083097, -0.024509646, 0.019968826, 0.0..."
2,None,[výpočetní technika],"[[0.009841297, -0.062487043, 0.04388702, 0.046..."
3,None,[politika],"[[0.067117184, 0.002262696, -0.0105991345, 0.0..."


In [120]:
class UserProfileManager(DBManager):
    
    def __init__(self, connection):
        super().__init__(connection)
        self._load_query = 'select * from interest_item'
        
    def loadFromDB(self, parts=10):
        print("Running query: " + self._load_query)
        raw_data = self.runQuery(self._load_query)
        
        user_profile_items = {}
        total_items = len(raw_data)
        print("Loading total " + str(total_items) + " items")
        for i, item in enumerate(raw_data):
            if i % (int(total_items / parts)+1) == 0:
                print("Progress: {}%".format(numpy.ceil(i * 100 / total_items)))
            item_id = item[0]
            user_id = item[1]
            item_desc = item[2]
            lembedding = item[3]
            embedding = numpy.array(lembedding)
            user_profile = user_profile_items.get(user_id, {'user_id':user_id, 'interest_items':[],'embeddings':[]})
            user_profile['interest_items'].append(item_desc)
            user_profile['embeddings'].append(embedding)
            user_profile_items[user_id] = user_profile
        return pandas.DataFrame(user_profile_items.values())
    
    def _truncateDB(self):
        self.runQuery('truncate table interest_item')
        self.runQuery('truncate table user_profile cascade')

    def saveToDB(self, df_user_profile):
        self._truncateDB()
        for index, row in df_user_profile.iterrows():
            user_id = self.runQuery('insert into user_profile default values returning user_id;')
            print(user_id)
            interest_items = row['interest_items']
            embeddings = row['embeddings']

            for i, item in enumerate(interest_items):
                print('    '+item)
                embedding = embeddings[i]
                lembedding = embedding.tolist()
                cursor = self._connection.cursor()

                postgres_insert_query = """INSERT INTO interest_item (user_id, item_desc, embedding)
                                            VALUES (%s,%s,%s)"""
                record_to_insert = (user_id, item, lembedding)
                cursor.execute(postgres_insert_query, record_to_insert)

                self._connection.commit()
                count = cursor.rowcount
                cursor.close()
                
upmngr = UserProfileManager(psycopg2_conn)

In [121]:
df_user_profile = upmngr.loadFromDB()
df_user_profile

Running query: select * from interest_item
Loading total 5 items
Progress: 0.0%
Progress: 20.0%
Progress: 40.0%
Progress: 60.0%
Progress: 80.0%


,user_id,interest_items,embeddings
0,9,[automobily],"[[-0.0220996141433716, -0.109728731215, 0.1329..."
1,10,"[silniční komunikace, stavby]","[[-0.0160830970853567, -0.0245096459984779, 0...."
2,11,[výpočetní technika],"[[0.00984129682183266, -0.0624870434403419, 0...."
3,12,[politika],"[[0.0671171844005585, 0.00226269592531025, -0...."


In [61]:
upmngr.saveToDB(df_user_profile)

9
    automobily
10
    silniční komunikace
    stavby
11
    výpočetní technika
12
    politika


In [22]:
df_contracts['item_cnt'] = df_contracts['subject_items'].apply(lambda items: len(items))
df_contracts['item_cnt'].sum()

In [98]:
vectors = []
vec_to_contr = []
for index, row in df_contracts.iterrows():
    for i, e in enumerate(row['embedding']):
        vectors.append(e)
        vec_to_contr.append((index, i))
nvectors = numpy.array(vectors)
nvec_to_contr = numpy.array(vec_to_contr)

In [99]:
nvec_to_contr[:,0]

array([  2,   2,   2, ..., 845, 845, 845])

In [114]:
target = embedder.embed('silnice')
target

array([-0.0322157 ,  0.01005193,  0.0531302 ,  0.04453046,  0.02474517,
        0.03930113,  0.03863908,  0.02281518, -0.04255066, -0.04347493,
       -0.03446029,  0.02713962, -0.02250312,  0.10116725,  0.00802387,
       -0.02142047, -0.01011338,  0.03464705, -0.04680473,  0.01213157,
        0.03080197,  0.04592355, -0.07544671,  0.07730675,  0.07038305,
       -0.11222419, -0.04332665, -0.02213423,  0.03521576, -0.04161568,
       -0.06193833, -0.01849777,  0.09476501, -0.03743394, -0.0334427 ,
       -0.00787411,  0.03515019,  0.02882177, -0.07536472,  0.14629231,
        0.01493124, -0.01861195,  0.042471  , -0.02584997,  0.11133599,
        0.10647292, -0.04358091, -0.0026977 ,  0.02174981, -0.0298345 ,
        0.04020512,  0.02210816,  0.00643665, -0.031235  , -0.00654671,
        0.08393693,  0.01202219, -0.0389263 , -0.11419459, -0.03638755,
        0.04249454,  0.01357855, -0.01816026, -0.06411272,  0.10980565,
       -0.02127203,  0.07787605,  0.09588517,  0.07617583, -0.03

In [115]:
start_time = time.time()
myvals = numpy.dot(nvectors, target)
top_indexes = numpy.argsort(myvals)[::-1][:10]
max_similarity = myvals[top_indexes]
results = []
for index in top_indexes:
    df_index, i = nvec_to_contr[index]
    contr_items = df_contracts.loc[df_index, 'subject_items']
    top_contr_result = contr_items[i]
    results.append({'top_result': top_contr_result, 'items': contr_items})
# max_similarity = df_contracts.loc[vec_to_contr[max_index],'subject_items']
end_time = time.time()
print("done with max dot in %s seconds" % (end_time - start_time) )
print("Most similar vector to target is index %s with %s" % (max_index, max_similarity))
print("Results are:")
display(results)

done with max dot in 0.0019638538360595703 seconds
Most similar vector to target is index 1232 with [0.49618003 0.49618003 0.49618003 0.4334278  0.4334278  0.41649634
 0.41649634 0.40689367 0.40689367 0.3987221 ]
Results are:


[{'top_result': 'Silnice II/290 Roprachtice, rekonstrukce silnice',
  'items': ['Silnice II/290 Roprachtice, rekonstrukce silnice',
   'Silnice II/290 Roprachtice, rekonstrukce silnice',
   'Silnice II/290 Roprachtice, rekonstrukce silnice',
   'SILNICE II / 290 Roprachtice, rekonstrukce silnice',
   'rekonstrukce silnice II/290 mezi městem Vysoké nad Jizerou a napojením na silnici I',
   'v obnově povrchu se zvýšením nivelety o max. 30 mm',
   'šest zastávek linkových autobusů']},
 {'top_result': 'Silnice II/290 Roprachtice, rekonstrukce silnice',
  'items': ['Silnice II/290 Roprachtice, rekonstrukce silnice',
   'Silnice II/290 Roprachtice, rekonstrukce silnice',
   'Silnice II/290 Roprachtice, rekonstrukce silnice',
   'SILNICE II / 290 Roprachtice, rekonstrukce silnice',
   'rekonstrukce silnice II/290 mezi městem Vysoké nad Jizerou a napojením na silnici I',
   'v obnově povrchu se zvýšením nivelety o max. 30 mm',
   'šest zastávek linkových autobusů']},
 {'top_result': 'Silnice I

In [95]:
start_time = time.time()
myvals = numpy.dot(nvectors, target)
max_index = numpy.argmax(myvals)
max_similarity = myvals[max_index]

results = []
df_index, i = nvec_to_contr[max_index]
contr_items = df_contracts.loc[df_index, 'subject_items']
top_contr_result = contr_items[i]
results.append({'top_result': top_contr_result, 'items': contr_items})
end_time = time.time()
print("done with max dot in %s seconds" % (end_time - start_time) )
print("Most similar vector to target is index %s with %s" % (max_index, max_similarity))
print("Results are:")
display(results)

done with max dot in 0.000965118408203125 seconds
Most similar vector to target is index 1232 with 0.32639343
Results are:


[{'top_result': 'AVACOM CarMAX 2',
  'items': ['AVACOM CarMAX 2', 'zdroj - AVACOM CarMAX 2', '']}]

In [54]:
nvectors = numpy.random.normal(0,100,(35196,300))
target = numpy.random.normal(0,100,(300))

In [80]:
target = numpy.array(embedder.process(['plyn','hrušky']))
target

array([[-2.11298210e-03, -1.82957441e-01,  5.43357581e-02,
         3.68261561e-02, -2.87125707e-02,  5.07773012e-02,
        -1.76154040e-02,  1.68402586e-03,  1.31049259e-02,
        -7.74341226e-02, -3.82218882e-02, -8.23296327e-03,
        -3.83996256e-02,  6.80910796e-02, -2.72737276e-02,
        -2.83118756e-03,  4.68822978e-02,  9.90544632e-02,
        -7.25169852e-03, -9.92601272e-03, -7.76487887e-02,
         6.20452501e-02, -4.20110002e-02, -4.40651271e-03,
         3.66706029e-02,  4.29575257e-02,  3.24910507e-03,
        -7.27823302e-02,  9.13716033e-02,  9.50580314e-02,
         4.21908386e-02,  5.82099408e-02, -1.56948995e-02,
        -4.12731282e-02, -5.98687753e-02,  3.88855487e-02,
        -5.31117246e-02, -1.57179423e-02, -5.97531758e-02,
         1.01783291e-01, -5.41335568e-02, -3.94700356e-02,
        -2.85113584e-02,  7.49275014e-02, -8.19963869e-03,
         9.70901623e-02, -6.23237640e-02,  3.86645943e-02,
        -3.53591517e-02,  1.30062653e-02, -3.70422751e-0

In [64]:
items = ['výpočetní technika', 'dodávka elektrické energie', 'školení logistiky', 'jablka', 'stavba školy']

In [73]:
vectors = numpy.array(embedder.process(items))
vectors

array([[ 0.0098413 , -0.06248704,  0.04388702, ...,  0.00350117,
         0.03128854,  0.00917289],
       [ 0.0353285 , -0.03633739,  0.06467913, ...,  0.01631052,
         0.01822275, -0.01730084],
       [ 0.07292239, -0.03028919,  0.08900586, ...,  0.01053857,
         0.1177141 ,  0.02247322],
       [ 0.04248842, -0.07491765,  0.04933689, ..., -0.01439078,
         0.01115297, -0.07369979],
       [ 0.07771003, -0.08573899,  0.09131815, ..., -0.00548958,
         0.07527085,  0.03874234]], dtype=float32)

In [83]:
start_time = time.time()
myvals = numpy.dot(vectors, target)
max_index = numpy.argmax(myvals)
max_similarity = myvals[max_index]
end_time = time.time()
print("done with max dot in %s seconds" % (end_time - start_time) )
print("Most similar vector to target is index %s with %s" % (max_index, max_similarity))

ValueError: shapes (5,300) and (2,300) not aligned: 300 (dim 1) != 2 (dim 0)

In [75]:
vectors.shape

(5, 300)

In [82]:
target.T.shape

(300, 2)

In [86]:
similarities = numpy.dot(vectors, target.T)
display(similarities.shape)
similarities

(5, 2)

array([[0.16289525, 0.09437177],
       [0.34905615, 0.13015456],
       [0.13001427, 0.14490783],
       [0.13005623, 0.50360495],
       [0.10587461, 0.13807026]], dtype=float32)

In [90]:
similarities = numpy.dot(target, vectors.T)
display(similarities.shape)
similarities

(2, 5)

array([[0.16289525, 0.34905615, 0.13001427, 0.13005623, 0.10587461],
       [0.09437177, 0.13015456, 0.14490783, 0.50360495, 0.13807026]],
      dtype=float32)

In [99]:
sorted_sims = numpy.argsort(similarities).T[::-1].T
sorted_sims

array([[1, 0, 3, 2, 4],
       [3, 2, 4, 1, 0]], dtype=int64)

In [103]:
selected_index = sorted_sims[:,:2]

In [104]:
[[(index, similarities[i, index])for index in row] for i, row in enumerate(selected_index)]

[[(1, 0.34905615), (0, 0.16289525)], [(3, 0.50360495), (2, 0.14490783)]]

In [157]:
class SimilarVectorComputer:
    
    def computeMostSimilar(self, target, vectors, num_results=1):
        similarities = numpy.dot(target, vectors.T)
        sorted_index = numpy.argsort(similarities).T[::-1].T
        selected_index = sorted_index[:,:num_results]
        return [[(index, similarities[i, index])for index in row] for i, row in enumerate(selected_index)]

In [158]:
sic = SimilarVectorComputer()
sic.computeMostSimilar(target, vectors, 2)

[[(1, 0.34905615), (0, 0.16289525)], [(3, 0.50360495), (2, 0.14490783)]]

In [124]:
df_user_profile

,user_id,interest_items,embeddings
0,9,[automobily],"[[-0.0220996141433716, -0.109728731215, 0.1329..."
1,10,"[silniční komunikace, stavby]","[[-0.0160830970853567, -0.0245096459984779, 0...."
2,11,[výpočetní technika],"[[0.00984129682183266, -0.0624870434403419, 0...."
3,12,[politika],"[[0.0671171844005585, 0.00226269592531025, -0...."


In [125]:
df_contract_items

,contract_id,subject_items,embedding
0,2,[krmení zvěře jádrovým a granulovým krmivem s ...,"[[0.00463546579703689, -0.079517237842083, 0.0..."
1,68,"[otovitel se zavazuje, že vyzve objednatele k ...","[[-0.00496369041502476, -0.0264993589371443, 0..."
2,23,[otovitel je povinen uchovávat veškerou dokume...,"[[-0.0100010493770242, -0.0357969366014004, 0...."
3,3,"[zhotovení díla, a to:. část č. 1. Oprava oplo...","[[-0.00123070902191103, -0.0510847456753254, 0..."
4,26,"[, VRSTVY PRO OBNOVU A OPRAVY Z ASF BETONU ACO...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...
154,839,"[Nové biotechnologické produkty ÚEB AVČR, reg....","[[-0.0160277131944895, -0.0321324989199638, 0...."
155,840,"[Nové biotechnologické produkty ÚEB AVČR, dodá...","[[-0.0254124347120523, -0.0627986714243889, 0...."
156,841,"[Nové biotechnologické produkty UEB AVČR, chem...","[[-0.0470611900091171, -0.0829258784651756, 0...."
157,842,"[Nové biotechnologické produkty ÚEB AVČR, chem...","[[-0.0254124347120523, -0.0627986714243889, 0...."


In [126]:
df_user_profile

,user_id,interest_items,embeddings
0,9,[automobily],"[[-0.0220996141433716, -0.109728731215, 0.1329..."
1,10,"[silniční komunikace, stavby]","[[-0.0160830970853567, -0.0245096459984779, 0...."
2,11,[výpočetní technika],"[[0.00984129682183266, -0.0624870434403419, 0...."
3,12,[politika],"[[0.0671171844005585, 0.00226269592531025, -0...."


In [143]:
contract_items = [{'contr_id':1, 'subject_items':['výpočetní technika', 'počítače']},
 {'contr_id':2, 'subject_items':['elektrická energie', 'plyn']},
 {'contr_id':3, 'subject_items':['školení logistiky', 'dopravník']},
 {'contr_id':4, 'subject_items':['jablka', 'hrušky']},
 {'contr_id':5, 'subject_items':['stavba školy']}]
df_contract_items = pandas.DataFrame(contract_items)
df_contract_items['embeddings'] = df_contract_items['subject_items'].apply(lambda items: embedder.process(items))
df_contract_items

,contr_id,subject_items,embeddings
0,1,"[výpočetní technika, počítače]","[[0.009841297, -0.062487043, 0.04388702, 0.046..."
1,2,"[elektrická energie, plyn]","[[0.009689108, -0.06226731, 0.033098727, 0.016..."
2,3,"[školení logistiky, dopravník]","[[0.072922386, -0.030289192, 0.08900586, 0.039..."
3,4,"[jablka, hrušky]","[[0.042488422, -0.07491765, 0.04933689, 0.0934..."
4,5,[stavba školy],"[[0.07771003, -0.08573899, 0.091318145, 0.0090..."


In [247]:
class SimilarItemsComputer:
    
    def __init__(self, df_contract_items):
        self._df_contract_items = df_contract_items
        self._nvectors, self._nvec_to_contr = self._count_mapping(df_contract_items)
        self._similar_vec_comp = SimilarVectorComputer()

    def _count_mapping(self, df_items):
        vectors = []
        vec_to_entity = []
        for index, row in df_items.iterrows():
            for i, e in enumerate(row['embeddings']):
                vectors.append(e)
                vec_to_entity.append((index, i))
        nvectors = numpy.array(vectors)
        nvec_to_entity = numpy.array(vec_to_entity)
        return nvectors, nvec_to_entity
        
    def computeMostSimilar(self, df_user_profile, num_results=1):
        target_nvectors, nvec_to_user_profile = self._count_mapping(df_user_profile)
        most_similar_vecs = self._similar_vec_comp.computeMostSimilar(target_nvectors, self._nvectors, num_results)
        
        results = {}
        
        for index_target_item, target_item_row in enumerate(most_similar_vecs):
            index_df_user_profile = nvec_to_user_profile[index_target_item][0]
            user_id = df_user_profile.loc[index_df_user_profile, 'user_id']
            interest_items = df_user_profile.loc[index_df_user_profile, 'interest_items']
            index_interest_item = nvec_to_user_profile[index_target_item][1]
            interest_item = interest_items[index_interest_item]
            
            user_results = results.get(user_id, {})
            results[user_id] = user_results
            item_results = user_results.get(interest_item, [])
            user_results[interest_item] = item_results
            
            for index_contr_item, similarity in target_item_row:
                index_df_contr = self._nvec_to_contr[index_contr_item][0]
                contr_id = self._df_contract_items.loc[index_df_contr, 'contr_id']
                contr_items = self._df_contract_items.loc[index_df_contr, 'subject_items']
                index_contr_item = self._nvec_to_contr[index_contr_item][1]
                contr_item = contr_items[index_contr_item]
                
                item_results.append({'contr_id':contr_id, 'contr_item': contr_item, 'similarity': similarity})
        return results

In [248]:
sic = SimilarItemsComputer(df_contract_items)
nvectors, nvec_to_contr = sic._nvectors, sic._nvec_to_contr
nvectors.shape

(9, 300)

In [249]:
nvec_to_contr

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1],
       [2, 0],
       [2, 1],
       [3, 0],
       [3, 1],
       [4, 0]])

In [250]:
target_nvectors, nvec_to_user_profile = sic._count_mapping(df_user_profile)
target_nvectors.shape

(5, 300)

In [251]:
nvec_to_user_profile

array([[0, 0],
       [1, 0],
       [1, 1],
       [2, 0],
       [3, 0]])

In [253]:
result = sic.computeMostSimilar(df_user_profile, 2)
result

{9: {'automobily': [{'contr_id': 3,
    'contr_item': 'dopravník',
    'similarity': 0.4554494514952735},
   {'contr_id': 1,
    'contr_item': 'počítače',
    'similarity': 0.31516830502811966}]},
 10: {'silniční komunikace': [{'contr_id': 3,
    'contr_item': 'dopravník',
    'similarity': 0.4080262246544052},
   {'contr_id': 3,
    'contr_item': 'školení logistiky',
    'similarity': 0.27377673952080883}],
  'stavby': [{'contr_id': 5,
    'contr_item': 'stavba školy',
    'similarity': 0.4827694182979704},
   {'contr_id': 3,
    'contr_item': 'dopravník',
    'similarity': 0.2578761377412154}]},
 11: {'výpočetní technika': [{'contr_id': 1,
    'contr_item': 'výpočetní technika',
    'similarity': 0.7281190126838999},
   {'contr_id': 1,
    'contr_item': 'počítače',
    'similarity': 0.46495545524119386}]},
 12: {'politika': [{'contr_id': 1,
    'contr_item': 'výpočetní technika',
    'similarity': 0.22716758406336263},
   {'contr_id': 3,
    'contr_item': 'školení logistiky',
    'si